In [7]:
n = 10**6 #кол-во необходимых случайных событий   

m = 2**31-1
a = 1220703125
c = 7
x_prev = 7

# Конгруэнтный метод
def generator():
    global x_prev
    x_prev = (a*x_prev + c) % m
    
    return x_prev / m


In [8]:
# Task 1: Иммитация простого случайного события
def generate_simple_event(p):
    if p < 0 or p > 1:
         raise Exception("Probability should be in range 0..1")
        
    num =  generator()
    
    if num < p:
        return True
        
    return False

def test_1(method_name, parameters):
    true_counter = 0
    false_counter = 0

    for i in range(n):
        if(method_name(parameters)):
            true_counter += 1
        else:
            false_counter += 1

    print(true_counter)
    print(false_counter)


test_1(generate_simple_event, 0.1)

100276
899724


In [9]:
# Task 2: Иммитация сложного события
def generator_complex_event(list_with_probabilities):
    result_with_generated_events = []
    
    for i in list_with_probabilities:
        result_with_generated_events.append(generate_simple_event(i))

    return result_with_generated_events


def test_2(method_name, parameters):
    arr = [[0 for j in range(2)] for i in range(len(parameters))]
    
    for i in range(n):
        arr_event_result = method_name(parameters)
        for j in range(len(arr_event_result)):
            if arr_event_result[j]:
                arr[j][0] += 1
            else:
                arr[j][1] += 1

    print(arr)


test_2(generator_complex_event, [0.1, 0.5, 0.4])

[[100243, 899757], [499850, 500150], [399217, 600783]]


In [10]:
# Task 4 : Иммитация событий, образующих полную группу
def generator_full_group_event(list_with_probabilities):
    arr_ranges = []
    prev = 0
    for i in list_with_probabilities:
        prev += i
        arr_ranges.append(prev)

    num = generator()
    
    for i in range(len(arr_ranges)):
        if num > arr_ranges[i]:
            continue
            
        return i


def test_4(method_name, param):
    arr_result = [0 for i in range(len(param))]
    
    for i in range(n):
        num = method_name(param)
        arr_result[num] += 1

    return arr_result
        
test_4(generator_full_group_event, [0.1, 0.4, 0.3, 0.2])

[100229, 400131, 299584, 200056]

In [11]:
# Task 3: Иммитация сложного события, состоящего из зависимых событий
def generator_depended_complex_event(a_probability, b_a_probability):
    # Find P(B)
    b_probability = b_a_probability
    arr_ranges = []
    arr_ranges.append(a_probability * b_probability) # P(AB)
    arr_ranges.append((1 - a_probability) * b_probability + arr_ranges[0]) # P(~AB)
    arr_ranges.append(a_probability * (1 - b_probability) + arr_ranges[1]) # P(A~B)
    arr_ranges.append((1 - a_probability) * (1 - b_probability) + arr_ranges[2]) # P(~A~B)

    num = generator()
    
    for i in range(len(arr_ranges)):
        if num > arr_ranges[i]:
            continue
            
        return i


def test_3(method_name, param):
    arr_result = [0 for i in range(4)]
    
    for i in range(n):
        num = method_name(param[0], param[1])
        arr_result[num] += 1

    return arr_result
    
test_3(generator_depended_complex_event, [0.1, 0.2])
    

[19926, 180415, 80088, 719571]

In [12]:
import math

list = ["Apple","Orange","Banana", "Apricot", "Avocado", "Coconut", "Durian", "Dragonfruit", "Fig","Gooseberry"]
list_r = ["Mango","Grape", "Guava", "Jackfruit", "Plum", "Kiwifruit", "Kumquat", "Lemon", "Lime", "Watermelon"]
list_u = ["Blueberry", "Raspberry", "Cranberry", "Strawberry", "Blackberry","Boysenberry","Mulberry"]


def generate_set(p_r=0.08, p_u=0.02):
    result_of_generator = generator_depended_complex_event(p_r, p_u)
    result_set_of_cards = []
    result_set_number = None # 0-simple 1-with rare 2-with unique
    
    # Исключаем возможность ультра-редкой и редкой одновременно
    while(result_of_generator == 0):
        result_of_generator = generator_depended_complex_event(p_r, p_u)
    
    if result_of_generator == 1:
        index = math.floor(generator() * len(list_u))
        result_set_of_cards.append(list_u[index])
        result_set_number = 2
    elif result_of_generator == 2:
        index = math.floor(generator() * len(list_r))
        result_set_of_cards.append(list_r[index])
        result_set_number = 1
 
    for i in range(len(result_set_of_cards), 5):
        index = math.floor(generator() * len(list))
        result_set_of_cards.append(list[index])
        
        if(result_set_number is None):
            result_set_number = 0

    return result_set_of_cards, result_set_number


def test():
    arr_statistics = [0] * 3
    
    for i in range(n):
        _, set_number = generate_set()
        arr_statistics[set_number] += 1
        
    print(arr_statistics)


test()

[903135, 78595, 18270]
